In [1]:
import bia_bob
import random

In [2]:
def prompt_scadsai_llm(message:str, model="openai/gpt-oss-120b", temperature=0.2):
    """A prompt helper function that sends a message to ScaDS.AI LLM server at 
    ZIH TU Dresden and returns only the text response.
    """
    import os
    import openai
    
    # convert message in the right format if necessary
    if isinstance(message, str):
        message = [{"role": "user", "content": message}]
    
    # setup connection to the LLM
    client = openai.OpenAI(base_url="https://llm.scads.ai/v1",
                           api_key=os.environ.get('SCADSAI_API_KEY')
    )
    response = client.chat.completions.create(
        model=model,
        messages=message,
        temperature=temperature
    )
    
    # extract answer
    return response.choices[0].message.content

prompt_scadsai_llm("hello world")

'Hello! 👋 How can I help you today?'

„Die Erstellung erfolgt unter Verwendung der maschinenlesbaren Fassung des Bundesinstituts für Arzneimittel und Medizinprodukte (BfArM).“
Source: https://www.bfarm.de/SharedDocs/Downloads/DE/Kodiersysteme/klassifikationen/icd-10-gm/version2025/icd10gm2025syst-meta_zip.html?nn=841246&cms_dlConfirm=true&cms_calledFromDoc=841246


In [3]:
import pandas as pd

# Load CSV with ";" as delimiter
df = pd.read_csv('icd10gm2025syst_kodes.csv', delimiter=';', header=None)
df = df[df[6].str.startswith("F")][[6, 8]]
df["diagnosis"] = df[6] + " " + df[8]
df

,6,8,diagnosis
2790,F00,Demenz bei Alzheimer-Krankheit,F00 Demenz bei Alzheimer-Krankheit
2791,F00.0,"Demenz bei Alzheimer-Krankheit, mit frühem Beg...","F00.0 Demenz bei Alzheimer-Krankheit, mit früh..."
2792,F00.1,"Demenz bei Alzheimer-Krankheit, mit spätem Beg...","F00.1 Demenz bei Alzheimer-Krankheit, mit spät..."
2793,F00.2,"Demenz bei Alzheimer-Krankheit, atypische oder...","F00.2 Demenz bei Alzheimer-Krankheit, atypisch..."
2794,F00.9,"Demenz bei Alzheimer-Krankheit, nicht näher be...","F00.9 Demenz bei Alzheimer-Krankheit, nicht nä..."
...,...,...,...
3304,F98.8,Sonstige näher bezeichnete Verhaltens- und emo...,F98.8 Sonstige näher bezeichnete Verhaltens- u...
3305,F98.80,Aufmerksamkeitsstörung ohne Hyperaktivität mit...,F98.80 Aufmerksamkeitsstörung ohne Hyperaktivi...
3306,F98.88,Sonstige näher bezeichnete Verhaltens- und emo...,F98.88 Sonstige näher bezeichnete Verhaltens- ...
3307,F98.9,Nicht näher bezeichnete Verhaltens- oder emoti...,F98.9 Nicht näher bezeichnete Verhaltens- oder...


In [4]:
import pandas as pd
import numpy as np
import random

# Define separate lists for first names and last names
first_names = ["Alex", "Jordan", "Morgan", "Taylor", "Avery",
               "Sam", "Casey", "Riley", "Jamie", "Dakota",
               "Bailey", "Cameron", "Sydney", "Robin", "Charlie",
               "Dana", "Kris", "Skyler", "Devon", "Reese"]

last_names = ["Reed", "Smith", "Lee", "Garcia", "Patel",
              "Chen", "O'Hara", "Jain", "Kumar", "Singh",
              "Liu", "Davis", "Clark", "Thomas", "Brooks",
              "Flores", "Rivera", "Adams", "Gonzalez", "Campbell"]

faculties = df["diagnosis"].tolist()
len(faculties), faculties[:10]

(519,
 ['F00 Demenz bei Alzheimer-Krankheit',
  'F00.0 Demenz bei Alzheimer-Krankheit, mit frühem Beginn (Typ 2)',
  'F00.1 Demenz bei Alzheimer-Krankheit, mit spätem Beginn (Typ 1)',
  'F00.2 Demenz bei Alzheimer-Krankheit, atypische oder gemischte Form',
  'F00.9 Demenz bei Alzheimer-Krankheit, nicht näher bezeichnet',
  'F01 Vaskuläre Demenz',
  'F01.0 Vaskuläre Demenz mit akutem Beginn',
  'F01.1 Multiinfarkt-Demenz',
  'F01.2 Subkortikale vaskuläre Demenz',
  'F01.3 Gemischte kortikale und subkortikale vaskuläre Demenz'])

In [6]:
research_fields = faculties

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

profiles = []

for field in faculties:
    # Random name generation from first and last name
    name = f"{random.choice(first_names)} {random.choice(last_names)}"
    #field = random.choice(research_fields)

    topic = prompt_scadsai_llm(f"""Generiere einen kurzen Arztbrieftext für jemanden, der Diagnose '{field}' hat. Der Arztbrief sollte nur eine Beschreibung enthalten, und die Diagnose selbst nicht. Antworte mit dem Arztbrief in 2-3 Sätzen und sonst nichts.""", temperature=0.9)
    topic = topic.strip().replace("</s>", "").replace("\"", "")
    if ":" in topic and " " not in topic.split(":")[0]:
        topic = topic.split(":")[1]
    if "</thinking>" in topic:
        topic = topic.split("</thinking>")[1]

    print(len(topic), topic)
    
    profiles.append({
        "name": name,
        "research_field": field,
        "topic": topic
    })

451 Der Patient zeigt seit mehreren Monaten eine fortschreitende Verschlechterung des Kurzzeitgedächtnisses, Schwierigkeiten bei der Orientierung in bekannten Umgebungen und wiederkehrende Vergesslichkeit von Alltagstätigkeiten. Zusätzlich treten Sprachfindungsstörungen sowie eine zunehmende Verwirrtheit in ungewohnten Situationen auf. Diese Symptome beeinträchtigen seine Selbstständigkeit erheblich und erfordern eine umfassende betreuende Versorgung.
456 Der Patient weist seit kurzer Zeit zunehmende Gedächtnisstörungen, Desorientierung und Schwierigkeiten bei alltäglichen Aktivitäten auf, wobei insbesondere das Kurzzeitgedächtnis und die Planungsfähigkeiten betroffen sind. Die kognitive Verschlechterung hat sich rasch entwickelt und beeinträchtigt zunehmend die Selbstständigkeit. Es wird eine engmaschige neurologische Betreuung, kognitive Therapiemaßnahmen und unterstützende Hilfen im Alltag empfohlen.
415 Der Patient weist fortschreitende kognitive Einschränkungen mit anhaltender Ged

In [8]:
# Create DataFrame and save CSV
df = pd.DataFrame(profiles)
df.to_csv("patients_diagnoses.csv", index=False)

df.head()

,name,research_field,topic
0,Taylor Reed,F00 Demenz bei Alzheimer-Krankheit,Der Patient zeigt seit mehreren Monaten eine f...
1,Jamie Jain,"F00.0 Demenz bei Alzheimer-Krankheit, mit früh...",Der Patient weist seit kurzer Zeit zunehmende ...
2,Riley Patel,"F00.1 Demenz bei Alzheimer-Krankheit, mit spät...",Der Patient weist fortschreitende kognitive Ei...
3,Taylor Adams,"F00.2 Demenz bei Alzheimer-Krankheit, atypisch...",Der Patient zeigt seit mehreren Monaten eine f...
4,Morgan Gonzalez,"F00.9 Demenz bei Alzheimer-Krankheit, nicht nä...",Bei der Patientin zeigen sich fortschreitende ...
